In [1]:
import sys

# install kfp (https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.html)
!{sys.executable} -m pip install --user --upgrade -q kfp==0.4.0

# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold && mv skaffold /home/jupyter/.local/bin/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44.6M  100 44.6M    0     0   135M      0 --:--:-- --:--:-- --:--:--  135M


In [2]:
# !{sys.executable} -m pip install --user --upgrade -q tfx==0.22.0
!python3 -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

TFX version: 0.22.0


In [3]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [4]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
print("GCP project ID:" + GCP_PROJECT_ID)

GCP project ID:nitroml-brain-xgcp


# Set KFP Cluster End point

In [5]:
# This refers to the KFP cluster endpoint
ENDPOINT='ee1a2cabbbc2f13-dot-us-east1.pipelines.googleusercontent.com' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [6]:
# Docker image name for the pipeline image 
IMAGE_NAME = 'nitroml_benchmark2'
CUSTOM_TFX_IMAGE='gcr.io/' + GCP_PROJECT_ID + '/' + IMAGE_NAME

In [10]:
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"), "AIHub" , 'nitroml', 'examples')
%cd {PROJECT_DIR}

/home/jupyter/AIHub/nitroml/examples


In [17]:
from examples import config
PIPELINE_NAME=config.PIPELINE_NAME

In [19]:
PIPELINE_NAME

'titanic_benchmark'

## Run the tfx pipeline

In [13]:
!tfx pipeline create  \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
Target image gcr.io/nitroml-brain-xgcp/nitroml_benchmark2 is not used. If the build spec is provided, update the target image in the build spec file build.yaml.
Use skaffold to build the container image.
/home/jupyter/.local/bin/skaffold
New container image is built. Target image is available in the build spec file.
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
I0626 12:51:31.939515 139867575141760 dataset_info.py:361] Load dataset info from /home/jupyter/tensorflow_datasets/titanic/2.0.0
I0626 12:51:31.941812 139867575141760 tfds_dataset.py:46] Preparing dataset...
I0626 12:51:31.942105 139867575141760 dataset_builder.py:282] Reusing dataset titanic (/hom

In [54]:
!tfx run create --pipeline-name='nitroml_benchmark' --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: nitroml_benchmark
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: nitroml_benchmark
+-------------------+--------------------------------------+----------+---------------------------+-------------------------------------------------------------------------------------------------------------------------+
| pipeline_name     | run_id                               | status   | created_at                | link                                                                                                                    |
+===================+======================================+==========+===========================+=========================================================================================================================+
| nitroml_benchmark | 383a4e61-9b39-4633-a247-cf605c0abdfd |          | 2020-06-25T23:55:17+00:00 | http://ee1a2cabbbc2f13-dot-us-east1.pipelines.googleuse

In [35]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_dag_runner.py \
--endpoint={ENDPOINT}

CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
Use skaffold to build the container image.
/home/jupyter/.local/bin/skaffold
New container image is built. Target image is available in the build spec file.
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
I0626 15:07:04.555761 139815216985472 nitroml.py:493] KUBEFLOW_TFX_IMAGE: gcr.io/nitroml-brain-xgcp/nitroml_benchmark@sha256:675febfd333ed463e316ccdf8f096c8035b174904d690b5bd9e5835755b78d6f
I0626 15:07:04.768827 139815216985472 dataset_info.py:361] Load dataset info from gs://artifacts.nitroml-brain-xgcp.appspot.com/tensorflow-datasets/titanic/2.0.0
I0626 15:07:05.428758 139815216985472 tfds_dataset.py:46] Preparing dataset...
I0626 15:07:05.463959 13981521

In [34]:
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: titanic_benchmark
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: titanic_benchmark
+-------------------+--------------------------------------+----------+---------------------------+-------------------------------------------------------------------------------------------------------------------------+
| pipeline_name     | run_id                               | status   | created_at                | link                                                                                                                    |
+===================+======================================+==========+===========================+=========================================================================================================================+
| titanic_benchmark | f6ec35ce-861f-4913-8595-4248e36614d3 |          | 2020-06-26T14:27:08+00:00 | http://ee1a2cabbbc2f13-dot-us-east1.pipelines.googleuse

In [37]:
tfx_image = os.environ.get('KUBEFLOW_TFX_IMAGE', None)

In [38]:
print (tfx_image)

None
